In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cheese-data-processed/new_train_x.pkl
/kaggle/input/cheese-data-processed/train_y.pkl
/kaggle/input/cheese-data-processed/new_test_x.pkl
/kaggle/input/cheese-data-processed/test_y.pkl


# Let's import the cheese data that was processed in EDA notebook

In [2]:
new_train_x = pd.read_pickle("/kaggle/input/cheese-data-processed/new_train_x.pkl")
new_test_x = pd.read_pickle("/kaggle/input/cheese-data-processed/new_test_x.pkl")
train_y = pd.read_pickle("/kaggle/input/cheese-data-processed/train_y.pkl")
test_y = pd.read_pickle("/kaggle/input/cheese-data-processed/test_y.pkl")

In [3]:
# Let's have a look at the data
new_train_x

,MoisturePercent,Organic,ManufacturerProvCode_BC,ManufacturerProvCode_ON,ManufacturerProvCode_QC,ManufacturingTypeEn_Artisan,ManufacturingTypeEn_Farmstead,ManufacturingTypeEn_Industrial,MilkTypeEn_Cow,MilkTypeEn_Ewe,...,CategoryTypeEn_Firm Cheese,CategoryTypeEn_Fresh Cheese,CategoryTypeEn_Hard Cheese,CategoryTypeEn_Semi-soft Cheese,CategoryTypeEn_Soft Cheese,CategoryTypeEn_Veined Cheeses,ManufacturerProvCode_others,MilkTypeEn_others,RindTypeEn_Rind,MilkTreatmentType_Pasteurized
CheeseId,,,,,,,,,,,,,,,,,,,,,
1556,0.45,0,0,0,1,1,0,0,1,0,...,0,0,0,1,0,0,0,0,1,1
794,0.38,0,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1493,0.48,0,0,0,1,1,0,0,1,0,...,0,0,0,1,0,0,0,0,1,1
1296,0.50,0,0,0,1,0,0,1,1,0,...,0,0,0,0,1,0,0,0,1,1
1324,0.40,0,0,0,1,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,0.43,0,0,0,1,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1
1130,0.45,0,0,0,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1332,0.56,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1


In [4]:
# Let's 100 more rows to the the training data since the dataset is small

new_train_x = pd.concat([new_train_x, new_test_x.iloc[:100]])
train_y = pd.concat([train_y, test_y.iloc[:100]])
new_test_x.drop(index=new_test_x.index[:100], inplace=True)
test_y.drop(index=test_y.index[:100], inplace=True)

In [5]:
new_train_x.shape, train_y.shape, new_test_x.shape, test_y.shape

((910, 23), (910,), (109, 23), (109,))

# Now let's train a model and make predictions!!

### We will be using Scikit Learn to train a model and use necessary packages in it as required, so let's go!!

## Let's train a Support Vector classifier

In [6]:
from sklearn.svm import SVC 

In [7]:
model = SVC(C=100, kernel="rbf", degree=1)

In [8]:
# Let's plot the learning curve while training

from sklearn.model_selection import LearningCurveDisplay

In [9]:
model.fit(new_train_x, train_y)

SVC(C=100, degree=1)

In [10]:
model.score(new_train_x, train_y)

0.8615384615384616

In [11]:
# we achived an accurecy of 87%
#Let's predict on the test set and check the accurecy

In [12]:
model.predict(new_test_x)

array([0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1])

In [13]:
model.score(new_test_x, test_y)

0.8348623853211009

In [14]:
# So we are overfitting, let's use randomized search CV to find the best params

from sklearn.model_selection import GridSearchCV

In [15]:
model2 = SVC(kernel = "rbf")

In [16]:
# Let's search for the best value of regularization parameter C and degree
distributions = {
                "C":[20,40,60,80,100],
                "degree":[1,2,3]
                 }

In [17]:
# Let's use accuracy as the scoring criteria and us 5 fold cross validation 

rs_clf = GridSearchCV(model2, distributions, scoring='accuracy', cv=5)

In [18]:
rs_clf.fit(new_train_x, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [20, 40, 60, 80, 100], 'degree': [1, 2, 3]},
             scoring='accuracy')

In [19]:
rs_clf.best_estimator_

SVC(C=40, degree=1)

In [20]:
rs_clf.best_score_

0.7945054945054946

In [21]:
rs_clf.predict(new_test_x)

array([0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1])

In [22]:
rs_clf.score(new_test_x, test_y)

0.8348623853211009

In [23]:
from sklearn.metrics import precision_score, recall_score

# Make predictions on the test set
y_pred = rs_clf.predict(new_test_x)

# Assuming new_test_y contains the true labels
precision = precision_score(test_y, y_pred)
recall = recall_score(test_y, y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Precision: 0.717948717948718
Recall: 0.8


In [24]:
# so we get a precision of 71.79% and Recall of 80%